In [1]:
# %pip install cvxpy

     |████████████████████████████████| 2.7 MB 512 kB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 7.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 215 kB 37.7 MB/s eta 0:00:01
     |████████████████████████████████| 921 kB 33.1 MB/s eta 0:00:01
  Created wheel for ecos: filename=ecos-2.0.7.post1-cp38-cp38-linux_x86_64.whl size=194426 sha256=0eed8580d823d02850c4cb541bbc1fbe47fdd694bad10d5af4b0665e66703946
  Stored in directory: /home/fellner/.cache/pip/wheels/c3/c1/ce/b551b0b3a40c990e1da2ec9aecff62f7c2d43cab8766368e82
  Created wheel for scs: filename=scs-2.1.4-cp38-cp38-linux_x86_64.whl size=432399 sha256=80c1fda546da12a212609fe961e2aa0bc02ffa3bf2fe4d84a372b1eb1197dfaa
  Stored in directory: /home/fellner/.cache/pip/wheels/f9/1b/79/5e399d671778e7a7770bc72053a60cac6c

In [1]:
import cvxpy as cp
import pandas as pd

# wget https://calmcode.io/datasets/stigler.csv
df = pd.read_csv("./../stigler.csv")
df

,commodity,unit,price_cents,calories,protein_g,calcium_g,iron_mg,vitamin_a_iu,vitamin_b1_mg,vitamin_b2_mg,niacin_mg,vitamin_c_mg
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,1 lb.,14.1,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,8 oz.,7.1,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8 oz.,16.2,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,10 lb.,51.7,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,24 oz.,13.7,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,18 oz.,13.6,9.0,0,10.3,244,0.0,1.9,7.5,146,0


In [2]:
price = df['price_cents'].values

x = cp.Variable(price.shape[0])

constraint_dict = {
    'protein_g': 70, 'calcium_g': 0.8, 'iron_mg': 12,
    'vitamin_a_iu': 5, 'calories': 2000,
    'vitamin_b1_mg': 1.8, 'vitamin_b2_mg': 2.7,
    'niacin_mg': 18, 'vitamin_c_mg': 75
}

objective = cp.Minimize(cp.sum(price@x))
constraints = [x >= 0]
for key, value in constraint_dict.items():
    constraints.append(x@df[key] >= value)
    
prob = cp.Problem(objective, constraints)
prob.solve()

255.60260040725825

In [3]:
(df.assign(assignment = x.value).
 sort_values(['assignment'], ascending=False)
 .head(10))

,commodity,unit,price_cents,calories,protein_g,calcium_g,iron_mg,vitamin_a_iu,vitamin_b1_mg,vitamin_b2_mg,niacin_mg,vitamin_c_mg,assignment
4,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0,5.555455e+01
45,Cabbage,1 lb.,3.7,2.6,125,4.0,36,7.2,9.0,4.5,26,5369,1.396908e-02
23,Lard,1 lb.,9.8,41.7,0,0.0,0,0.2,0.0,0.5,5,0,5.781545e-09
68,"Navy Beans, Dried",1 lb.,5.9,26.9,1691,11.4,792,0.0,38.4,24.6,217,0,1.076392e-09
49,Onions,1 lb.,3.6,5.8,166,3.8,59,16.6,4.7,5.9,21,1184,8.081185e-10
52,Sweet Potatoes,1 lb.,5.1,9.6,138,2.7,54,290.7,8.4,5.4,83,1912,6.723142e-10
7,Rolled Oats,1 lb.,7.1,25.3,907,5.1,341,0.0,37.1,8.9,64,0,4.064024e-10
40,Apples,1 lb.,4.4,5.8,27,0.5,36,7.3,3.6,2.7,5,544,3.391856e-10
46,Carrots,1 bunch,4.7,2.7,73,2.8,43,188.5,6.1,4.3,89,608,3.355819e-10
66,"Peas, Dried",1 lb.,7.9,20.0,1367,4.2,345,2.9,28.7,18.4,162,0,3.143655e-10
